In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment, StreamTableEnvironment
from pyflink.datastream import StreamExecutionEnvironment, RuntimeExecutionMode
import os
from pyflink.common import Types, Row
from pyflink.datastream.connectors.jdbc import JdbcSink, JdbcConnectionOptions, JdbcExecutionOptions

In [2]:
RUNTIME_ENV = os.getenv("RUNTIME_ENV", "local")
BOOTSTRAP_SERVERS = os.getenv("BOOTSTRAP_SERVERS", "localhost:9092")

CURRENT_DIR = os.getcwd()
jar_files = [
    "flink-sql-connector-postgres-cdc-3.1.1.jar",
    "postgresql-42.6.0.jar", 
    "flink-connector-jdbc-3.1.2-1.18.jar"
]
jar_urls = [f"file:///{CURRENT_DIR}/Downloads/{jar_file}" for jar_file in jar_files]

jar_paths = tuple(
            [f"file://{os.path.join(CURRENT_DIR, 'Downloads', name)}" for name in jar_files]
        )

# Set up the JAR files first
jars = ";".join(jar_urls)
os.environ["PYFLINK_CLIENT_JAR"] = jars

In [3]:
env = StreamExecutionEnvironment.get_execution_environment()
env.add_jars(*jar_paths)
table_env = StreamTableEnvironment.create(env)
env.set_runtime_mode(RuntimeExecutionMode.STREAMING)

JavaObject id=o62

In [4]:
table_env.get_config().get_configuration().set_string("pipeline.jars", jars)

In [5]:
postgres_sink = f"""
CREATE TABLE vehicle_updates_cdc_test (
    vehicle_id STRING,
    vehicle_label STRING,
    trip_id STRING,
    start_date DATE,
    latitude FLOAT,
    longitude FLOAT,
    bearing INT,
    updated_at TIMESTAMP,
    occupancy_status INT,
    occupancy_percentage INT
 ) WITH (
  'connector' = 'postgres-cdc',
   'hostname' = 'localhost',
   'port' = '5432',
   'username' = 'root',
   'password' = 'root',
   'database-name' = 'transitstreamtest',
   'schema-name' = 'public',
   'table-name' = 'vehicle_position_updates',
   'slot.name' = 'vehicleupdatescdc',
   'decoding.plugin.name' = 'pgoutput'
 );
"""

table_env.execute_sql(postgres_sink)

In [7]:
def addRowKind(row):
    # vehicle_id = row.vehicle_id
    # vehicle_label = row.vehicle_label
    # trip_id = row.trip_id
    # start_date = row.
    op = str(row.get_row_kind())
    return Row(*row, op)

row_type_info = Types.ROW([Types.STRING(), Types.STRING(), Types.STRING(), Types.SQL_DATE(), Types.FLOAT(), Types.FLOAT(), Types.INT(), Types.SQL_TIMESTAMP(), Types.INT(), Types.INT(), Types.STRING()])
res_table = table_env.sql_query("""
           select *
           from vehicle_updates_cdc_test
           """)

table_env.to_changelog_stream(res_table).map(addRowKind, row_type_info).add_sink(
    JdbcSink.sink(
        """INSERT INTO public.vehicle_updates_cdc_test (vehicle_id, vehicle_label, trip_id, start_date, latitude, longitude, bearing, updated_at, occupancy_status, occupancy_percentage, op)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        row_type_info, 
        JdbcConnectionOptions.JdbcConnectionOptionsBuilder()
            .with_url('jdbc:postgresql://localhost:5432/transitstreamtest')
            .with_driver_name('org.postgresql.Driver')
            .with_user_name('root')
            .with_password('root')
            .build(),
        JdbcExecutionOptions.builder()
            .with_batch_size(1)
            .with_max_retries(0)
            .build()
)
)
    
    
env.execute()

KeyboardInterrupt: 